In [1]:
import sys

## Part 1 - Figure out what customer ids appear multiple times - this will reduce the memory we need later

The plan is that this will enable us to handle most of the lines of the file easily. Since, it is only resource intensive where need to handle custtomer ids that appear mutliple times.

In [2]:
line_no = 0
all_cust_ids = set()   # will contain all unique cust_ids
multi_cust_ids = set()   # will contain cust_ids that appear multiple times
prod_cats = ['Wireless', 'Books', 'Digital_Ebook_Purchase', 'Video DVD',
       'Electronics', 'Digital_Music_Purchase', 'Home', 'Shoes',
       'Digital_Video_Download', 'Musical Instruments', 'Camera',
       'Kitchen', 'Baby', 'Music', 'Home Improvement', 'Beauty',
       'Office Products', 'Video', 'Automotive', 'Apparel', 'Jewelry',
       'Outdoors', 'PC', 'Toys', 'Sports', 'Health & Personal Care',
       'Lawn and Garden', 'Tools', 'Major Appliances',
       'Home Entertainment', 'Video Games', 'Watches', 'Pet Products',
       'Luggage', 'Grocery', 'Mobile_Apps', 'Digital_Software',
       'Furniture', 'Software', 'Mobile_Electronics', 'Gift Card',
       'Digital_Video_Games', 'Personal_Care_Appliances']

# read the file line by line avoiding loading entire file into memory
for line in open('/media/paul/Data/datasets/DSPC_reviews_association_mining/raw_transactions_file.csv'):
    # keep track of line no
    line_no = line_no + 1
    
    # There is an encoding error after line 124343000 hence stop before this (we are nearly at the end of the file anyway)
    if line_no == 124343000:
        break
    
    # print progress every 1,000,000 lines
    if ((line_no%1000000) == 0):
        print('Processing line ' + str(line_no) + '(' + str(round((line_no/124343000)*100,2)) + '%)')
    
    # break into cells
    cells = line.strip('').split(',')
    
    # Check line has expected number of cells
    if(len(cells) != 4):
        #print("Bad line: " + str(line_no) + " - incorrect number of cells")
        print("Xc,", end='')
        continue
        
    # Check the prod_cat is valid
    if(cells[1] not in prod_cats):
        #print("Bad line: " + str(line_no) + " - invalid product category")
        print("Xp,", end='')
        continue
    
    # read the customer id
    cust_id = cells[0]
    
    # check whether we have seen this id before?
    pre_add_length = len(all_cust_ids) # check the length
    
    # add the id to the set
    all_cust_ids.add(cust_id) 
    
    # if the id has been seen before then len would have not increased,
    # hence we know this cust_id is appearing for an another time.
    if pre_add_length == len(all_cust_ids):
        multi_cust_ids.add(cust_id)

del all_cust_ids

print("Found " + str(len(multi_cust_ids)) + " id appearing multiple reviews")
print(str(round(sys.getsizeof(multi_cust_ids) / 1000000000, 2)) + "GB")

Processing line 1000000(0.8%)
Processing line 2000000(1.61%)
Processing line 3000000(2.41%)
Xc,Processing line 4000000(3.22%)
Xc,Xp,Processing line 5000000(4.02%)
Xc,Xp,Xp,Xp,Processing line 6000000(4.83%)
Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Processing line 7000000(5.63%)
Xp,Xp,Xp,Xp,Processing line 8000000(6.43%)
Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Processing line 9000000(7.24%)
Processing line 10000000(8.04%)
Processing line 11000000(8.85%)
Processing line 12000000(9.65%)
Processing line 13000000(10.45%)
Processing line 14000000(11.26%)
Xp,Xp,Processing line 15000000(12.06%)
Processing line 16000000(12.87%)
Xp,Xp,Processing line 17000000(13.67%)
Processing line 18000000(14.48%)
Processing line 19000000(15.28%)
Processing line 20000000(16.08%)
Processing line 2100

## Part 2 - Handle the customer ids that appear once and prepare to handle to ids that appear multiple times

In [5]:
# list to contain handle the aggrgation when customers have purchased multi items
cust_id_to_prods = {}

# output file
f = open("/media/paul/Data/datasets/DSPC_reviews_association_mining/transactions_vertical.csv", "w")

# write the header
header = ''
for prod in prod_cats:
    header = header + prod + ','
header = header.strip(',')
f.write(header + '\n')

line_no = 0
# loop over file again, now processing only the cust_ids that appear once
for line in open('/media/paul/Data/datasets/DSPC_reviews_association_mining/raw_transactions_file.csv'):
    # keep track of line no
    line_no = line_no + 1
    
    # There is an encoding error after line 124343000 hence stop before this, (we are nearly at the end of the file anyway)
    if line_no == 124343000:
        break
        
    # print progress every 1,000,000 lines
    if ((line_no%1000000) == 0):
        print('Processing line ' + str(line_no) + '(' + str(round((line_no/124343000)*100,2)) + '%)')
    
    # break into cells
    cells = line.strip('').split(',')
    
    # Check line has expected number of cells
    if(len(cells) != 4):
        #print("Bad line: " + str(line_no) + " - incorrect number of cells")
        print("Xc,", end='')
        continue
        
    # Check the prod_cat is valid
    if(cells[1] not in prod_cats):
        #print("Bad line: " + str(line_no) + " - invalid product category")
        print("Xp,", end='')
        continue
    
    # read the customer id
    cust_id = cells[0]    
    prod = cells[1]
        
    # Handle the cases where they only bought one item, (easy, we have all the info we need)
    if cust_id not in multi_cust_ids:
        output_line = ''
        # write y for the product they bought and ? otherwise
        for prod_cat in prod_cats:
            if prod_cat == prod:
                output_line = output_line + 'y,'                
            else:
                output_line = output_line + '?,'
        output_line = output_line.strip(',') # remove unnecessary comma
        f.write(output_line + '\n')
    
    # Handle the cases where they bought multi items, (harder, we need to accumulate)
    else:
        # have we seen this cust_id before
        if cust_id in cust_id_to_prods.keys():
            # yes add to the set of prods
            cust_id_to_prods[cust_id].add(prod)
        else:
            # no create a a new set
            cust_id_to_prods[cust_id] = {prod}

f.close()
del multi_cust_ids

print("Aggregated the reviews of " + str(len(cust_id_to_prods)) + " customers")
print(str(round(sys.getsizeof(cust_id_to_prods) / 1000000000, 2)) + "GB")

Processing line 1000000(0.8%)
Processing line 2000000(1.61%)
Processing line 3000000(2.41%)
Xc,Processing line 4000000(3.22%)
Xc,Xp,Processing line 5000000(4.02%)
Xc,Xp,Xp,Xp,Processing line 6000000(4.83%)
Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Processing line 7000000(5.63%)
Xp,Xp,Xp,Xp,Processing line 8000000(6.43%)
Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Xp,Processing line 9000000(7.24%)
Processing line 10000000(8.04%)
Processing line 11000000(8.85%)
Processing line 12000000(9.65%)
Processing line 13000000(10.45%)
Processing line 14000000(11.26%)
Xp,Xp,Processing line 15000000(12.06%)
Processing line 16000000(12.87%)
Xp,Xp,Processing line 17000000(13.67%)
Processing line 18000000(14.48%)
Processing line 19000000(15.28%)
Processing line 20000000(16.08%)
Processing line 2100

## Part 3 - Handle complex cases, where customer ids appear multiple times

In [6]:
# append to output file
f = open("/media/paul/Data/datasets/DSPC_reviews_association_mining/transactions_vertical.csv", "a")

dict_size = len(cust_id_to_prods)

id_no = 0
for cust_id in cust_id_to_prods:
    id_no = id_no + 1
    
    # print progress every 100,000 ids
    if ((id_no%100000) == 0):
        print('(' + str(round((id_no/dict_size)*100,2)) + '%)')
    
    # write y for the product they bought and ? otherwise
    output_line = ''
    for prod_cat in prod_cats:        
        if prod_cat in cust_id_to_prods[cust_id]:
            output_line = output_line + 'y,'
        else:
            output_line = output_line + '?,'
    output_line = output_line.strip(',') # remove unnecessary comma
    f.write(output_line + '\n')

f.close()
del cust_id_to_prods

print("Complete!")

Complete!
